In [1]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import os

print("✅ Libraries loaded!")

Mounted at /content/drive
✅ Libraries loaded!


In [2]:
model = tf.keras.models.load_model(
    '/content/drive/MyDrive/Pneumonia_Project/Models/saved_models/densenet121_best_model.keras'
)
print("✅ DenseNet121 model loaded!")

✅ DenseNet121 model loaded!


In [7]:
# FIXED - space instead of underscore
EXTERNAL_PATH = '/content/drive/MyDrive/Pneumonia_Project/Independent Dataset 	/test'

print("NORMAL exists:", os.path.exists(f'{EXTERNAL_PATH}/NORMAL'))
print("PNEUMONIA exists:", os.path.exists(f'{EXTERNAL_PATH}/PNEUMONIA'))
print()
print("NORMAL images:", len(os.listdir(f'{EXTERNAL_PATH}/NORMAL')))
print("PNEUMONIA images:", len(os.listdir(f'{EXTERNAL_PATH}/PNEUMONIA')))

NORMAL exists: True
PNEUMONIA exists: True

NORMAL images: 237
PNEUMONIA images: 251


In [6]:
import os

# Search for the folder
base = '/content/drive/MyDrive/Pneumonia_Project'

for root, dirs, files in os.walk(base):
    for d in dirs:
        if 'normal' in d.lower() or 'pneumonia' in d.lower() or 'independent' in d.lower() or 'test' in d.lower():
            print(os.path.join(root, d))

/content/drive/MyDrive/Pneumonia_Project/Independent Dataset 	
/content/drive/MyDrive/Pneumonia_Project/Data/test
/content/drive/MyDrive/Pneumonia_Project/Data/test/NORMAL
/content/drive/MyDrive/Pneumonia_Project/Data/test/PNEUMONIA
/content/drive/MyDrive/Pneumonia_Project/Data/train/NORMAL
/content/drive/MyDrive/Pneumonia_Project/Data/train/PNEUMONIA
/content/drive/MyDrive/Pneumonia_Project/Data/val/NORMAL
/content/drive/MyDrive/Pneumonia_Project/Data/val/PNEUMONIA
/content/drive/MyDrive/Pneumonia_Project/Independent Dataset 	/test
/content/drive/MyDrive/Pneumonia_Project/Independent Dataset 	/test/NORMAL
/content/drive/MyDrive/Pneumonia_Project/Independent Dataset 	/test/PNEUMONIA


In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)

external_gen = test_datagen.flow_from_directory(
    EXTERNAL_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

print("🔍 Running predictions on external dataset...")
y_pred_prob = model.predict(external_gen).flatten()
y_true = external_gen.classes

# Use your optimized threshold from DenseNet121
THRESHOLD = 0.260
y_pred = (y_pred_prob > THRESHOLD).astype(int)

cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

accuracy    = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision   = tp / (tp + fp)
f1          = 2 * (precision * sensitivity) / (precision + sensitivity)

print()
print("=" * 60)
print("EXTERNAL DATASET VALIDATION RESULTS")
print("=" * 60)
print(f"Total Samples:  {len(y_true)}")
print(f"NORMAL:         {sum(y_true == 0)}")
print(f"PNEUMONIA:      {sum(y_true == 1)}")
print()
print(f"Accuracy:       {accuracy*100:.2f}%")
print(f"Sensitivity:    {sensitivity*100:.2f}%")
print(f"Specificity:    {specificity*100:.2f}%")
print(f"Precision:      {precision*100:.2f}%")
print(f"F1-Score:       {f1:.4f}")
print()
print(f"Confusion Matrix:")
print(f"  TN: {tn}  FP: {fp}")
print(f"  FN: {fn}  TP: {tp}")
print("=" * 60)

Found 488 images belonging to 2 classes.
🔍 Running predictions on external dataset...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 171s 10s/step

EXTERNAL DATASET VALIDATION RESULTS
Total Samples:  488
NORMAL:         237
PNEUMONIA:      251

Accuracy:       87.09%
Sensitivity:    97.21%
Specificity:    76.37%
Precision:      81.33%
F1-Score:       0.8857

Confusion Matrix:
  TN: 181  FP: 56
  FN: 7  TP: 244


In [9]:
import os

report = f"""
======================================================================
DENSENET121 - EXTERNAL DATASET VALIDATION REPORT
======================================================================

Date: 2026-02-25
Model: DenseNet121 (trained on Kaggle Pediatric Chest X-Ray)
External Dataset: Pneumonia Radiography Dataset (Kaggle)
Source: ayushirathour/chest-xray-pneumonia-detection-ai

PURPOSE:
----------------------------------------------------------------------
Cross-dataset validation to test model generalization.
The model was trained on Kaggle Pediatric dataset and tested
on a completely independent dataset it has never seen before.

EXTERNAL TEST SET:
----------------------------------------------------------------------
Total Samples:  {len(y_true)}
NORMAL:         {sum(y_true == 0)}
PNEUMONIA:      {sum(y_true == 1)}

RESULTS (Threshold = 0.260):
----------------------------------------------------------------------
Accuracy:       {accuracy*100:.2f}%
Sensitivity:    {sensitivity*100:.2f}%
Specificity:    {specificity*100:.2f}%
Precision:      {precision*100:.2f}%
F1-Score:       {f1:.4f}

CONFUSION MATRIX:
----------------------------------------------------------------------
True Negatives:  {tn}
False Positives: {fp}
False Negatives: {fn}
True Positives:  {tp}

COMPARISON - Kaggle vs External Dataset:
----------------------------------------------------------------------
Metric          Kaggle Test     External Dataset
Accuracy        94.31%          {accuracy*100:.2f}%
Sensitivity     95.01%          {sensitivity*100:.2f}%
Specificity     92.44%          {specificity*100:.2f}%

CLINICAL INTERPRETATION:
----------------------------------------------------------------------
This cross-dataset validation demonstrates the model's ability
to generalize beyond its training distribution, which is critical
for real-world clinical deployment in Algerian hospitals.

======================================================================
"""

os.makedirs('/content/drive/MyDrive/Pneumonia_Project/Models/results',
            exist_ok=True)

save_path = '/content/drive/MyDrive/Pneumonia_Project/Models/results/external_validation_report.txt'

with open(save_path, 'w', encoding='utf-8') as f:
    f.write(report)

print("✅ Report saved to Drive!")
print(f"📁 {save_path}")

✅ Report saved to Drive!
📁 /content/drive/MyDrive/Pneumonia_Project/Models/results/external_validation_report.txt
